Лабораторная работа - тренировка ЛЛМ на датасете [DocVQA](https://rrc.cvc.uab.es/?ch=17&com=introduction). <br>
Датасет содержит анотации с вопросами к документам и OCR по словам.

Будем тренировать [phi-2](https://huggingface.co/microsoft/phi-2) - 2.7b params <br>
Т.к мы ограничены по ресурсам, будем тренировать две модели 8bit и 4bit и сравним их результаты

In [8]:
import os
import json

import numpy as np

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model  
from accelerate import FullyShardedDataParallelPlugin, Accelerator

import torch
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

from datasets import load_dataset

from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

### 2. Тренировка модели

In [ ]:
dataset = load_dataset('json', 
                        data_files={'train': '/kaggle/input/docvqa/annotations_train.json',
                                    'val': '/kaggle/input/docvqa/annotations_val.json'}, field="data")

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers', 'data_split', 'context'],
        num_rows: 39463
    })
    val: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers', 'data_split', 'context'],
        num_rows: 5349
    })
})

In [11]:
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
# Load model
model_path = "microsoft/phi-2"

model = AutoModelForCausalLM.from_pretrained(
    model_path,    
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_8bit=True,
    ),
    torch_dtype=torch.bfloat16,        
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
    trust_remote_code=True,
    use_fast=False, # needed for now, should be fixed soon
)
tokenizer.pad_token = tokenizer.eos_token

In [16]:
def prepare_prompt(sample):
    full_prompt =f"""Context:{sample['context']} Instruction: Answer question using context.
### Question:
{sample["question"]}
### Answer:
{sample["answers"][0]}
"""
    sample['prompt'] = full_prompt
    return sample

In [ ]:
remove_columns=["context", "question", "question_types", "image", "docId", "ucsf_document_id", "ucsf_document_page_no", "data_split"]
dataset = dataset.map(prepare_prompt, 
                    remove_columns=remove_columns)

In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['questionId', 'answers', 'prompt'],
        num_rows: 39463
    })
    val: Dataset({
        features: ['questionId', 'answers', 'prompt'],
        num_rows: 5349
    })
})

In [19]:
max_length = 2048
def tokenize_function(example):
    return tokenizer(example["prompt"],
                     truncation=True,
                     max_length=max_length,
                     padding="max_length")

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, remove_columns=dataset["train"].column_names)

In [21]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 39463
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5349
    })
})

In [22]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) 

# Adapter settings
lora_config = LoraConfig(
    r=32, 
    lora_alpha=32, 
    target_modules = [ "q_proj", "k_proj", "v_proj", "dense" ],
    modules_to_save = ["lm_head", "embed_tokens"],
    lora_dropout=0.1, 
    bias="none", 
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

model.config.use_cache = False

In [23]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [24]:
project = "docvqa_8bit_150steps"
base_model_name = "phi2"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

train_args=TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        max_steps=150,
        learning_rate=2.5e-5,
        logging_steps=5,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        
        save_strategy="steps",       
        save_steps=50,                              
        report_to="tensorboard",           
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          
    )

In [25]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset['train'],
    args=train_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False 

In [ ]:
trainer.train()

![Phi2 Trian loss](./img/docvqa_train_losses.jpg)

In [ ]:
def prepare_val_prompt(sample):
    full_prompt =f"""Context:{sample['context']} Instruction: Answer question using context.
### Question:
{sample["question"]}
### Answer:
"""
    sample['prompt'] = full_prompt
    return sample

In [ ]:
validation_ds = dataset['val'].map(prepare_val_prompt)

In [ ]:
answers = []
model.eval()
with open('phi2-150steps-8bit.json', 'w') as f:
    for idx, item in enumerate(validation_ds):
        prompt = tokenizer(item['prompt'], return_tensors="pt").to('cuda')
        generated_output = model.generate(**prompt, use_cache=True, max_new_tokens=10)
        cur_dict = {}
        cur_dict['output'] = tokenizer.batch_decode(generated_output)[0].split('### Answer:\n')[1]
        cur_dict['answers'] = item['answers']
        cur_dict['questionId'] = item['questionId']
        answers.append(cur_dict)
    json.dump(answers, fp=f)

Примеры сгенерированного текста:

Context:UNIVERSITY OF CALIFORNIA, SAN DIEGO To Date Time WHILE YOU WERE OUT Mr. Ms. From Telephoned Jasips Clinic [ Will phone again [.Please phone OCame to see you [] Will come again JRush MESSAGE Taken by "Barce https://www.industrydocuments.ucsf.edu/docs/nkbl0226    Instruction: Answer question using context. <br>
Question: <br>
What is name of university? <br>
Answer: <br>
University of California, San Diego

Качество ответов будем считать с помощью метрики ANLS с threshold = 0.5 и метрики Exact Match 

In [ ]:
import textdistance

def anls_score(prediction: str, labels: str, threshold: float = 0.5) -> float:
    y_pred = prediction.strip().lower()
    score = max([textdistance.levenshtein.normalized_similarity(y_pred, label.strip().lower())
                    for label in labels])
    if 1 - threshold >= score:
        score = 0 

    return score

def em_score(prediction: str, labels: str) -> float:
    y_pred = prediction.strip().lower()
    score = np.max([int(y_pred == label.strip().lower()) for label in labels])
    return score

In [ ]:
with open('exps/docvqa/phi2_outputs.json', 'r') as f, open('exps/docvqa/phi2_outptus.txt', 'w') as fout:
    preds = json.load(f)
    anls_scores = []
    exact_match = []
    for idx, item in enumerate(preds):
        anls_scores.append(anls_score(item['output'], item['answers']))
        exact_match.append(em_score(item['output'], item['answers']))
        
    print('ANLS: ', np.mean(anls_scores), '\nEM: ', np.mean(em_score), file=fout)

| model      | steps | Anls | Exact Match |
| ----------- | :-----------: | :-----------: | :-----------: |
| phi2-4bit     | 50 | 0.7329 | 0.6964 |
| phi2-4bit   | 100 | 0.7526 | 0.7261 |
| phi2-4bit     | 150 | 0.7785 | 0.7468 |
| phi2-8bit   | 50 | 0.7638 | 0.7131 |
| phi2-8bit      | 100 | 0.7781 | 0.7492 |
| phi2-8bit   | 150 | 0.7916 | 0.7512 |

Результаты модельки с квантизацией в 8bit очевидно получились лучше, но при это не сильно. В целом презультаты получились достаточно хорошие для модельки в 2.7B параметров и интовыми весами. 

Примеры сегенерированных ответов лежат в ./data/phi2_processed_outputs_exp.json